In [2]:
from tensorflow import keras
import numpy as np
import pandas as pd

dataRep = '../data/'
scriptRep = '../script/'

# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, scriptRep)
import progressBar

In [3]:
unicodeData = pd.read_csv(dataRep+'unicode_translation.csv')
raw = np.load('../data/dataset/caracterClassificationDatasetFull.npz')
image = np.expand_dims(raw['image'], 3)[:500]
print(image.shape[0], image.shape[1:])

charOutput = np.zeros((image.shape[0], len(unicodeData)), dtype=np.bool)

for i in range(500):
    progressBar.update_progress(float(i)/float(500))
    charOutput[i, unicodeData[unicodeData['char']==raw['caracter'][:500][i]].index.values.astype(int)[0]] = True

Progress: [####################] 99.8%


In [6]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=image.shape[1:]))
model.add(keras.layers.Conv2D(16, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(len(unicodeData), activation='softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 30, 30, 16)        160       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 16)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 32)        4640      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 10, 32)        9248      
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 3200)             

In [7]:
model.fit(image, charOutput, epochs=10)

Epoch 1/10


InvalidArgumentError: Can not squeeze dim[1], expected a dimension of 1, got 4787
	 [[{{node metrics_2/acc/Squeeze}}]]